In [30]:
#import important libraries
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

#Read data and sav in variable
data= pd.read_csv(r"D:\SQL & POWERBI PORTFOLIO PROJECT\Store_Sales\Orders.csv")
data.head(3)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


In [31]:
data['Order Date']=pd.to_datetime(data['Order Date'])
data['Ship Date']=pd.to_datetime(data['Ship Date'])

C:\Users\Akash Jha\AppData\Local\Temp\ipykernel_13484\1816349176.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Order Date']=pd.to_datetime(data['Order Date'])
C:\Users\Akash Jha\AppData\Local\Temp\ipykernel_13484\1816349176.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Ship Date']=pd.to_datetime(data['Ship Date'])


In [32]:
# Postal code has more than 80% we can drop them
data.dropna(axis=0,inplace=True)

# handle missing values
data.isnull().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
City              0
State             0
Country           0
Postal Code       0
Market            0
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
Shipping Cost     0
Order Priority    0
dtype: int64

In [33]:
# Add new column profit-to-sales-ratio

data['Profit_to_sales_ratio']=data['Profit']/data['Sales']

In [34]:
# create sales high and low column
def category_sales(sales):
    if sales > 500:
        return "High"
    return "low"
data['Sales_Category']=data["Sales"].apply(category_sales)

In [43]:
#Extract year & month from order date & ship date
data['Order_Year']=data['Order Date'].dt.year
data['Order_month']=data['Order Date'].dt.month
data['Ship_Year']=data['Ship Date'].dt.year
data['Ship_month']=data['Ship Date'].dt.month

# No use of making month columns drop
data = data.drop(['Order_month','Ship_month'],axis=1)

In [78]:
data['Order_month']=data['Order Date'].dt.month

In [50]:
data['Date_Diff'] = data['Ship Date'] - data ['Order Date']
data['Date_Diff'].mean()

Timedelta('8 days 00:41:12.523514108')

In [56]:
# Orders that have delay in shipping
delay_ship_item = data[data['Date_Diff'] > '8 days' ]
delay_ship_item['Date_Diff'].value_counts()

153 days    409
122 days    400
61 days     317
123 days    207
92 days     173
120 days    140
91 days     112
31 days      90
59 days      88
184 days     79
183 days     71
151 days     67
181 days     61
30 days      54
34 days      49
121 days     47
214 days     44
63 days      42
36 days      40
150 days     37
62 days      37
35 days      32
89 days      30
94 days      30
64 days      26
33 days      25
212 days     24
65 days      22
90 days      19
37 days      18
60 days      18
95 days      15
213 days     15
152 days     15
28 days      13
66 days      12
154 days     12
93 days      11
32 days      10
182 days      9
125 days      9
67 days       6
96 days       5
124 days      4
126 days      4
29 days       4
155 days      2
Name: Date_Diff, dtype: int64

## Analysis & learn functions

In [59]:
 # Function to calculate profit/margin
def calculate_profit_margin(row):
    return round((row['Profit'] / row['Sales']) * 100 if row['Sales'] != 0 else 0,2)

# Apply the function to each row of the DataFrame
data['Profit_margin_%']=data.apply(calculate_profit_margin,axis=1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Discount,Profit,Shipping Cost,Order Priority,Profit_to_sales_ratio,Sales_Category,Order_Year,Ship_Year,Date_Diff,Profit_margin_%
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,0.0,762.1845,933.57,Critical,0.330000,High,2012,2012,0 days,33.00
8,40155,CA-2014-135909,2014-10-14,2014-10-21,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,...,0.2,1906.4850,867.69,Low,0.375000,High,2014,2014,7 days,37.50
9,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,...,0.4,-1862.3124,865.74,Critical,-0.433333,High,2012,2012,3 days,-43.33
10,34577,CA-2011-102988,2011-05-04,2011-09-04,Second Class,GM-14695,Greg Maxwell,Corporate,Alexandria,Virginia,...,0.0,83.2810,846.54,High,0.020000,High,2011,2011,123 days,2.00
16,36178,CA-2014-143567,2014-03-11,2014-06-11,Second Class,TB-21175,Thomas Boland,Corporate,Henderson,Kentucky,...,0.0,517.4793,780.70,Critical,0.230000,High,2014,2014,92 days,23.00


In [66]:
#Always Save Original Data
data_copy= data.copy()

In [74]:
# category wise sales
def calculate_sales_category(df):
    return (df.groupby('Category')['Sales'].sum().sort_values(ascending=False).reset_index())

category_wise_sales = calculate_sales_category(data)
category_wise_sales

,Category,Sales
0,Technology,836154.0330
1,Furniture,741999.7953
2,Office Supplies,719047.0320


In [76]:
# Region wise sales
def calculate_Sales_region(df):
    return(df.groupby('Region')['Sales','Profit'].sum().sort_values(by='Sales',ascending=False).reset_index())

region_wise_sales_profit=calculate_Sales_region(data)
region_wise_sales_profit

C:\Users\Akash Jha\AppData\Local\Temp\ipykernel_13484\1089157489.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  return(df.groupby('Region')['Sales','Profit'].sum().sort_values(by='Sales',ascending=False).reset_index())


,Region,Sales,Profit
0,West,725457.8245,108418.4489
1,East,678781.2400,91522.7800
2,Central,501239.8908,39706.3625
3,South,391721.9050,46749.4303


In [82]:
# lets  analysis data with some functions to learn
# Monthly trends 
def calculate_monthly_trends(df):
    return (df.groupby(['Order_Year','Order_month'])['Sales'].sum().sort_values(ascending=False).reset_index())
monthly_sales_trends = calculate_monthly_trends(data)
monthly_sales_trends

,Order_Year,Order_month,Sales
0,2014,11,84924.4660
1,2014,3,80235.3360
2,2014,9,77790.2180
3,2014,10,76203.1362
4,2014,8,74492.3475
5,2013,12,74355.7580
6,2013,11,72985.8510
7,2011,12,67350.0867
8,2013,5,66946.5680
9,2012,9,66729.3290


In [86]:
#Top Products
def Calculate_top_products(df,n=10):
    top_products=df.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).reset_index()
    return top_products.nlargest(n,'Sales')

top_products_sales_wise=Calculate_top_products(data)
top_products_sales_wise


,Product Name,Sales
0,Canon imageCLASS 2200 Advanced Copier,61599.824
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.384
2,Cisco TelePresence System EX90 Videoconferenci...,22638.480
3,HON 5400 Series Task Chairs for Big and Tall,21870.576
4,GBC DocuBind TL300 Electric Binding System,19823.479
5,GBC Ibimaster 500 Manual ProClick Binding System,19024.500
6,Hewlett Packard LaserJet 3310 Copier,18839.686
7,HP Designjet T520 Inkjet Large Format Printer ...,18374.895
8,GBC DocuBind P400 Electric Binding System,17965.068
9,High Speed Automatic Electric Letter Opener,17030.312


In [79]:
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Market', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping Cost', 'Order Priority', 'Sales_Category', 'Order_Year',
       'Ship_Year', 'Date_Diff', 'Profit_margin_%', 'Order_month'],
      dtype='object')